[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/crewai/simple-getting-started.ipynb)

# CrewAI and Weaviate Getting Started Example

This notebook shows you how industry-specific agents use web search and Weaviate blogs to track trends and uncover AI use cases across biomedical, healthcare, and finance.

## Import Libraries and Dependencies

At the time of creating this notebook, we're using the `weaviate-client` version `4.15.4`, `crewai` and `crewai-tools` version `1.6.1`.

In [18]:
%uv pip install weaviate-client==4.15.4 crewai==1.6.1 crewai-tools==1.6.1

/opt/homebrew/Cellar/python@3.13/3.13.11/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=85670) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Using Python 3.13.11 environment at: venv
Resolved 155 packages in 626ms                                       
Uninstalled 1 package in 61ms
Installed 1 package in 6ms15.4                              
 - weaviate-client==4.19.2
 + weaviate-client==4.15.4
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv

from crewai_tools import WeaviateVectorSearchTool
import weaviate
from weaviate.classes.init import Auth

from crewai import Agent
from crewai_tools import WeaviateVectorSearchTool
from crewai import Task
from crewai import Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool

load_dotenv()


False

## Connect to Serper API and OpenAI

You can get 2,500 of free credits with Serper [here](https://serper.dev/)!

In [4]:
os.environ["SERPER_API_KEY"] = "15a09aee29dd95d99842692b60a3f4de83049064"
#os.environ["OPENAI_API_KEY"] = ""

## Connect to Weaviate

Create a free sandbox cluster on [Weaviate Cloud](https://console.weaviate.cloud/)!

In [5]:
WCD_CLUSTER_URL = "8yty7w5ws9mwt7aakuzja.c0.us-west3.gcp.weaviate.cloud"
WCD_CLUSTER_KEY = "RnIyT2p4d3hJSVhCQkZPaF8vWGVsM2EydHR2U05uenZMYUZQT1YxcktkSFA5MHdLZ0M2ZTdhMHpJSFdvPV92MjAw"

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WCD_CLUSTER_URL,
    auth_credentials=Auth.api_key(WCD_CLUSTER_KEY),
    headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")}
)

print(client.is_ready())

True


## Connect to Existing Weaviate Collection

You'll want to connect to your existing Weaviate collection. If you don't have one already, you can follow this import notebook [here](https://github.com/weaviate/recipes/blob/main/integrations/Weaviate-Import-Example.ipynb)!

In [6]:
weaviate_collection = client.collections.get("WeaviateBlogChunk")

## Initialize the Tools

### Weaviate Search Tool

In [7]:
weaviate_tool = WeaviateVectorSearchTool(
    collection_name='WeaviateBlogChunk', 
    limit=4,
    alpha=0.75,
    weaviate_cluster_url=WCD_CLUSTER_URL,
    weaviate_api_key=WCD_CLUSTER_KEY,
)

### Web Search Tool

In [8]:
search_tool = SerperDevTool()

## Create the Agents

### BioMed Marketing Agent

In [9]:
BiomedicalMarketingAgent = Agent(
    role='Industry researcher focused on biomedical trends and their applications in AI',
    goal='Continuously track the latest biomedical advancements and identify how Weaviate’s features can support AI applications in biomedical research, diagnostics, and personalized medicine.',
    backstory='As a former biomedical product marketer turned AI strategist, you understand the complex language and regulatory landscape of biomedical innovation. With a keen eye on genomics, clinical research, and medical devices, it now leverages LLMs and vector search to explore how Weaviate’s capabilities can streamline scientific discovery and patient-centric campaigns.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

### Healthcare Marketing Agent 

In [10]:
HealthcareMarketingAgent = Agent(
    role='AI-savvy marketer specializing in healthcare systems, digital health, and patient engagement.',
    goal='Stay updated on healthcare policy shifts, digital health trends, and explore how Weaviate’s features can optimize workflows in hospital systems, EHR integration, and health communication.',
    backstory='Rooted in public health communications, this agent has evolved into a digital health consultant. You focus on how retrieval-augmented generation (RAG), semantic search, and hybrid models can be applied to solve healthcare-specific challenges—from patient triage to clinical support systems.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

### Financial Marketing Agent

In [11]:
FinancialMarketingAgent = Agent(
    role='Insight analyst exploring innovations in finance, wealth tech, and regulatory tech',
    goal='Monitor financial sector trends including AI in trading, compliance automation, and client advisory, and assess how Weaviate’s tools can enable cutting-edge financial applications.',
    backstory='With experience at a fintech startup and a background in capital markets, this agent specializes in using structured + unstructured data to surface insights for analysts and advisors. Now, it’s looking into how vector databases and LLMs can automate tasks like fraud detection, investor personalization, and market research.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

## Define the Agents Tasks

### BioMed Agent Task

In [12]:
biomed_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = BiomedicalMarketingAgent
)

### Healthcare Agent Task

In [13]:
healthcare_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = HealthcareMarketingAgent
)

### Financial Agent Task

In [14]:
financial_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = FinancialMarketingAgent
)

### List out the Weaviate Features

In [15]:
weaviate_features = [
  {"weaviate_feature": "MUVERA"},
  {"weaviate_feature": "Multi-tenancy"},
  {"weaviate_feature": "Compliance"},
  {"weaviate_feature": "Hybrid Search"}
]

### Assemble the Crew

In [16]:
blog_crew = Crew(
    agents=[BiomedicalMarketingAgent, HealthcareMarketingAgent, FinancialMarketingAgent],
    tasks=[biomed_agent_task, healthcare_agent_task, financial_agent_task],
    # verbose=True, # uncomment if you'd like to see the full execution from the kickoff
    # planning=True
)

## Query Time

In [ ]:
result = blog_crew.kickoff_for_each(inputs=weaviate_features)

print(result)

<frozen importlib._bootstrap>:488: ResourceWarning: unclosed database in <sqlite3.Connection object at 0x1337eb100>
<frozen importlib._bootstrap>:488: ResourceWarning: unclosed database in <sqlite3.Connection object at 0x1339e4e50>
<frozen importlib._bootstrap>:488: ResourceWarning: unclosed database in <sqlite3.Connection object at 0x1339e5210>


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Industry researcher focused on biomedical trends and their applications in AI                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Conduct a thorough research about MUVERA                                                               │
│          Make sure you find any interesting and relevant information using the web and Weaviate blogs.          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Industry researcher focused on biomedical trends and their applications in AI                           │
│                                                                                                                 │
│  Thought: I need to gather comprehensive information about MUVERA, including its applications and relevance in  │
│  biomedical research. I will start by searching for recent and relevant information on the web.                 │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "MUVERA biomedical applications Weaviate"                                                    │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'MUVERA biomedical applications Weaviate', 'type': 'search', 'num': 10, 'engine':   │
│  'google'}, 'organic': [{'title': 'More efficient multi-vector embeddings with MUVERA - Weaviate', 'link':      │
│  'https://weaviate.io/blog/muvera', 'snippet': 'MUVERA converts multi-vector embeddings (like ColBERT/ColPali)  │
│  into single fixed-size vectors, dramatically reducing memory and computational ...', 'position': 1,            │
│  'sitelinks': [{'title': 'Muvera To The Rescue\u200b', 'link':                                                  │
│  'https://weaviate.io/blog/muvera#:~:text=MUVERA%20to%20the%20rescue%E2%80%8B'}, {'title': 'Space               │
│  Partitioning\u200b', 'link': 'https://weaviate.io/blog/muvera#:~:text=Space%20partitioning%E2%80%8B'},         │
│  {'title': 'Impact Of Muvera\u200b', 'link':                                                                    │
│  'https://weaviate.io/blog/muvera#:~:text=Impact%20of%20MUVERA%E2%80%8B'}]}, {'title': 'Building Multi-Agent    │
│  Systems with Crew AI and Weaviate', 'link': 'https://weaviate.io/blog/building-multi-agent-systems',           │
│  'snippet': 'Each agent uses two tools: a Weaviate vector search tool for retrieval from an existing            │
│  collection and the Serper web search tool for fresh ...', 'position': 2}, {'title': 'MUVERA with Rajesh        │
│  Jayaram and Roberto Esposito - Weaviate ...', 'link':                                                          │
│  'https://creators.spotify.com/pod/profile/weaviate/episodes/MUVERA-with-Rajesh-Jayaram-and-Roberto-Esposito--  │
│  -Weaviate-Podcast-123-e33fnpi', 'snippet': 'This podcast dives into all things IReRa, XMC, DSPy compilation,   │
│  and applications in Biomedical NLP and Recommendation! I hope you find this ...', 'position': 3}, {'title':    │
│  'MUVERA with Rajesh Jayaram and Roberto Esposito — Weaviate ...', 'link':                                      │
│  'https://connorshorten300.medium.com/muvera-with-rajesh-jayaram-and-roberto-esposito-weaviate-podcast-123-b8c  │
│  07076ac3d', 'snippet': 'This blog post unpacks how MUVERA provides a novel compression technique to make       │
│  sophisticated multi-vector retrieval efficient and scalable.', 'position': 4}, {'title': "Weaviate's Post -    │
│  LinkedIn", 'link': 'https://www.linkedin.com/posts/weaviate-io_multi-vector-e...                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "content": "---\ntitle: Hybrid Search Explained\nslug: hybrid-search-explained\nauthors: [erika]\ndate:      │
│  2023-01-03\ntags: ['concepts', 'search']\nimage: ./img/hero.png\ndescription: \"Learn about the new hybrid     │
│  search feature that enables you to combine dense and sparse vectors to deliver the best of both search         │
│  methods.\"\n---\n![Hybrid search](./img/hero.png)\n\n<!-- truncate -->\n\nHybrid search is a technique that    │
│  **combines** multiple search algorithms to improve the accuracy and relevance of search results. It uses the   │
│  best features of both keyword-based search algorithms with vector search techniques. By leveraging the         │
│  strengths of different algorithms, it provides a more effective search experience for users. The [Hybrid       │
│  search](/developers/weaviate/api/graphql/search-operators#hybrid) feature was introduced in Weaviate 1.17. It  │
│  uses sparse and dense vectors to represent the meaning and context of search queries and documents.",          │
│    "author": null                                                                                               │
│  }{                                                                                                             │
│    "content": "Weaviate automatically combines the results from all of the target search vectors. The           │
│  mechanism is conceptually similar to the [hybrid fusion](/blog/hybrid-search-fusion-algorithms) algorithm in   │
│  [hybrid search](/developers/weaviate/search/hybrid). This allows you to tune your multi vector search to       │
│  balance the target vectors in a way that reflects your application needs. For details, see [multi target       │
│  vector search](/developers/weaviate/search/multi-vector). ## Tenant states changed\n\nTenant [activity         │
│  status](/developers/weaviate/concepts/data#tenant-status) is used to manage resource usage.",                  │
│    "author": null                                                                                               │
│  }{                                                                                                             │
│    "content": "We have prepared [this short                                                                     │
│  survey](https://docs.google.com/forms/d/e/1FAIpQLSfxekC2eh5Xkl0iJYEAoOkfhK4FA5AO-Od3KUz4srreOpq00g/viewform?u  │
│  sp=sf_link). We would really appreciate your input. Please let us know what you think!\n\n\n\n##               │
│  Wrap-up\n\nHybrid search in Weaviate offers a powerful blend of vector and keyword search, using the           │
│  strengths of both to deliver semant...                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Insight analyst exploring innovations in finance, wealth tech, and regulatory tech                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Based on the information gathered, here is a comprehensive analysis of Hybrid Search and its significant       │
│  applications within the finance sector:                                                                        │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Industry-Specific Analysis of Hybrid Search and Its Applications in Finance                                │
│                                                                                                                 │
│  Hybrid Search technology, particularly as implemented in Weaviate, represents a groundbreaking advancement in  │
│  information retrieval that is poised to revolutionize the finance industry. By merging traditional keyword     │
│  search methodologies with semantic vector-based searches, Hybrid Search enhances the retrieval capabilities    │
│  needed in an industry characterized by vast amounts of data and the necessity for precision.                   │
│                                                                                                                 │
│  #### Key Features of Hybrid Search in Finance                                                                  │
│                                                                                                                 │
│  1. **Integration of Keyword and Semantic Searches**:                                                           │
│     Hybrid Search takes advantage of the complementary strengths of keyword-based algorithms and vector-based   │
│  search techniques. In the finance sector, this could mean simultaneously searching for documents related to    │
│  "capital markets" while also retrieving relevant discussions around associated topics like "investment         │
│  strategies" or "market trends." This dual capability ensures that financial professionals can access not only  │
│  specific reports but also contextually relevant insights that enhance decision-making.                         │
│                                                                                                                 │
│  2. **Enhanced Accuracy and Relevance**:                                                                        │
│     The effectiveness of Hybrid Search lies in its combined scoring mechanism that optimally fuses results      │
│  from both search methodologies. For example, financial analysts often sift through extensive research reports  │
│  or regulatory documents that may not contain precise keywords but are still highly relevant to their           │
│  inquiries. By merging results and achieving a finely-tuned balance between keyword precision and semantic      │
│  context, Hybrid Search allows professionals to hone in on critical information quickly.                        │
│                                                                                                                 │
│  3. **Application in Fraud Detection and Compliance**:                                                          │
│     In an era of increasing regulatory scrutiny and the

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): [CrewOutput(raw="Thought: I have successfully obtained insights about MUVERA from both the internet and Weaviate's documentation. Now, I can formulate an industry-specific analysis regarding the potential applications of MUVERA within the financial sector.\n\n### MUVERA: Revolutionizing Financial Data Retrieval with Multi-Vector Encoding\n\nThe evolving landscape of financial services is increasingly reliant on sophisticated data retrieval techniques, driven by the need for better analytics, risk management, and compliance processes. A significant innovation poised to enhance data handling within this sector is Weaviate’s MUVERA (Multi-Vector Retrieval via Fixed Dimensional Encodings), which optimizes multi-vector embeddings for rapid and efficient data retrieval. \n\n#### The Challenge of Data Complexity in Finance\n\nThe financial sector generates a vast array of data types, including transactional data, customer prof



╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                                                              │
│  Info: Tracing has been disabled.                                            │
│                                                                              │
│  Your preference has been saved. Future Crew/Flow executions will not        │
│  collect traces.                                                             │
│                                                                              │
│  To enable tracing later, do any one of these:                               │
│  • Set tracing=True in your Crew/Flow code                                   │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file               │
│  • Run: crewai traces enable                                                 │
│                                                                              │
╰─────────────────────────